In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas

In [ ]:
df = pandas.read_csv('results/combined-results.csv', sep = '\t')
df

In [ ]:
# PCM colors
pcm_color_python = '#1f77b4'
pcm_color_io500 = '#ff7f0e'
pcm_color_tpcc = '#2ca02c'
pcm_color_noop = '#d62728'

# Kill-a-Watt colors
kw_color_python = '#0c3048'
kw_color_io500 = '#6c3200'
kw_color_tpcc = '#124012'
kw_color_noop = '#561010'

# All available hatches
hatches = ['/', '\\', '|', '-', '+', 'x', 'o', 'O', '.', '*']

# Default labels and colors (PCM only)
base_labels = ["Python Build", "IO500", "TPC-C", "Noop (idle)"]
base_colors = [pcm_color_python, pcm_color_io500, pcm_color_tpcc, pcm_color_noop]

# All labels and colors (PCM and KW)
all_labels = ["Python (PCM)", "Python (KaW)", "IO500 (PCM)", "IO500 (KaW)", "TPC-C (PCM)", "TPC-C (KaW)", "Noop (PCM)", "Noop (KaW)"]
all_colors = [pcm_color_python, kw_color_python, pcm_color_io500, kw_color_io500, pcm_color_tpcc, kw_color_tpcc, pcm_color_noop, kw_color_noop]

In [ ]:
def get_workload_style_map(workloads):
    markers = ["o", "s", "^", "D"]
    style_map = {}

    for i, wl in enumerate(workloads):
        style_map[wl] = {
            "color": base_colors[i % len(base_colors)],
            "marker": markers[i % len(markers)],
        }

    return style_map

In [ ]:
seconds_in_hour = 1 * 60 * 60
watts_in_kilowatt = 1000

def to_kilowatt_hours(joules):
    return ((joules / watts_in_kilowatt )/ seconds_in_hour)

In [ ]:
cols = ["Python Raw Energy (Joules)", "IO500 Raw Energy (Joules)", "TPCC Raw Energy (Joules)", "Noop Raw Energy (Joules)"]

style_map = get_workload_style_map(cols)

# Line Chart: Seconds
fig, ax = plt.subplots(figsize=(9, 4.5))

df_plot = df.iloc[:251]
x = numpy.arange(len(df_plot))

for col, label in zip(cols, base_labels):
    style = style_map[col]
    ax.plot(
        x,
        df_plot[col],
        label=label,
        color=style["color"],
        marker=style["marker"],
        markevery=1,
        markersize=3,
        linewidth=2.0,
    )

step = 50
ax.set_xticks(range(0, len(df_plot), step))
ax.set_xticklabels(range(0, len(df_plot), step))

ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Energy Consumption (Joules)")
ax.set_title("Power Consumption Across Various Workloads")

ax.legend(ncol=2, frameon=False, bbox_to_anchor=(1.02, 1), loc="upper left")

plt.tight_layout()
fig.savefig("images/power_consumption_seconds.png", bbox_inches='tight')
plt.show()

In [ ]:
cols = ["Python Energy per Minute", "IO500 Energy per Minute", "TPCC Energy per Minute", "Noop Energy per Minute"]

style_map = get_workload_style_map(cols)

# Line Chart: Aggregate Data
fig, ax = plt.subplots(figsize=(9, 4.5))

df_minute = df.iloc[:df['Time (Minute)'].count()]
x = numpy.arange(len(df_minute))

for col, label in zip(cols, base_labels):
    style = style_map[col]
    ax.plot(
        x,
        df_minute[col],
        label=label,
        color=style["color"],
        # marker=style["marker"],
        # markevery=10,
        # markersize=3,
        linewidth=2.0,
    )

step = 50
ax.set_xticks(range(0, len(df_minute), step))
ax.set_xticklabels(range(0, len(df_minute), step))

ax.set_xlabel("Time (Minutes)")
ax.set_ylabel("Energy Consumption (Joules)")
ax.set_title("Power Consumption Across Various Workloads")

ax.legend(ncol=2, frameon=False, bbox_to_anchor=(1.02, 1), loc="upper left")

plt.tight_layout()
fig.savefig("images/power_consumption_minutes.png", bbox_inches='tight')
plt.show()

In [ ]:
cols = ["Python Raw Energy (Joules)", "IO500 Raw Energy (Joules)", "TPCC Raw Energy (Joules)", "Noop Raw Energy (Joules)"]

sums = df[cols].sum()

fig, ax = plt.subplots(figsize=(8, 5))

x = numpy.arange(len(cols))

bars = []
for i, (col, label) in enumerate(zip(cols, base_labels)):
    b = ax.bar(
        x[i],
        sums[col],
        color=base_colors[i],
        hatch=hatches[i],
        edgecolor="black",
        linewidth=1.2,
    )
    bars.append(b)

# Axis labels and title
ax.set_xticks(x)
ax.set_xticklabels(base_labels)
ax.set_ylabel("Total Energy Consumption (Joules)")
ax.set_title("Total Energy Consumption by Workload")

# Create legend using proxy artists
legend_patches = [
    matplotlib.patches.Patch(
        facecolor=base_colors[i],
        hatch=hatches[i],
        edgecolor="black",
        label=labels[i]
    )
    for i in range(len(cols))
]

ax.legend(handles=legend_patches, title="Workload", frameon=True)

plt.tight_layout()
fig.savefig("images/total_consumption_joules.png", bbox_inches='tight')
plt.show()

In [ ]:
kwh_data = {}
for i in range(len(cols)):
    total_joules = df[cols[i]].sum()
    kilowatt_hours = to_kilowatt_hours(total_joules)
    label = cols[i]
    label = label.replace("Raw Energy (Joules)", "(PCM)")
    label = label.replace("TPCC", "TPC-C")
    kwh_data[label] = kilowatt_hours

# Add in the results from the KaW monitoring
kwh_data["Python (KaW)"] = 0.09
kwh_data["IO500 (KaW)"] = 0.04
kwh_data["TPC-C (KaW)"] = 0.06
kwh_data["Noop (KaW)"] = 0.00

In [ ]:
kwh_data

In [ ]:
sums = df[cols].sum()

fig, ax = plt.subplots(figsize=(12, 5))

bars = []
for i, label in enumerate(all_labels):
    b = ax.bar(
        label,
        kwh_data[label],
        color=all_colors[i],
        hatch=hatches[i],
        edgecolor="black",
        linewidth=1.2,
    )
    bars.append(b)

for bar in bars:
    for rect in bar:
        height = rect.get_height()
        ax.annotate(f"{height:.3f}",
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),   # 3-pixel offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

# Axis labels and title
fig.autofmt_xdate(rotation=35)
# ax.set_xticks(x)
# ax.set_xticklabels(all_labels)

ax.set_ylim(0, 0.10)
ax.set_yticks(numpy.arange(0, 0.11, 0.01))

ax.set_ylabel("Total Energy Consumption (KiloWatt Hours)")
ax.set_title("Total Energy Consumption by Workload")

# Create legend using proxy artists
legend_patches = [
    matplotlib.patches.Patch(
        facecolor=all_colors[i],
        hatch=hatches[i],
        edgecolor="black",
        label=all_labels[i]
    )
    for i in range(len(all_labels))
]

ax.legend(handles=legend_patches, title="Workload", frameon=True)

plt.tight_layout()
fig.savefig("images/total_consumption_kwh.png", bbox_inches='tight')
plt.show()

In [ ]:
prefixes = ['Python', 'IO500', 'TPC-C', 'Noop']
error_rate = {}
for prefix in prefixes:
    pcm_label = "%s (PCM)" % (prefix)
    kw_label = "%s (KaW)" % (prefix)

    pcm_value = kwh_data[pcm_label]
    kw_value = kwh_data[kw_label]

    if kw_value == 0:
        error_rate[prefix] = 0
        continue

    error_rate[prefix] = (abs(pcm_value - kw_value) / kw_value)

error_rate

In [ ]:
error_labels = ["Python", "IO500", "TPC-C", "Noop"]

fig, ax = plt.subplots(figsize=(8, 5))

bars = []
for i, label in enumerate(error_labels):
    b = ax.bar(
        label,
        error_rate[label],
        color=base_colors[i],
        hatch=hatches[i],
        edgecolor="black",
        linewidth=1.2,
    )
    bars.append(b)

for bar in bars:
    for rect in bar:
        height = rect.get_height()
        ax.annotate(f"{height:.3f}",
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),   # 3-pixel offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

# Axis labels and title
fig.autofmt_xdate(rotation=35)
# ax.set_xticks(x)
# ax.set_xticklabels(all_labels)

ax.set_ylim(0, 1.0)
ax.set_yticks(numpy.arange(0, 1.1, 0.1))

ax.set_ylabel("Relative Error Rate")
ax.set_title("Relative Error Rate by Workload")

# Create legend using proxy artists
legend_patches = [
    matplotlib.patches.Patch(
        facecolor=base_colors[i],
        hatch=hatches[i],
        edgecolor="black",
        label=error_labels[i]
    )
    for i in range(len(error_labels))
]

ax.legend(handles=legend_patches, title="Workload", frameon=True)

plt.tight_layout()
fig.savefig("images/pcm_error_rate.png", bbox_inches='tight')
plt.show()